## Scrape the Location Counts

### 1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:
* Starbucks

* Dunkin’ Donuts

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

### 2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

## Supplemental Data